In [2]:
from search_utils import web_search

import os

os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

result = web_search("what is banana", search_engine='bing', max_results=3)
# print(web_search("Sodium magnesium sulfate", search_engine='wikipedia', max_results=3))#, search_site='wikipedia.org'))
print(len(result))#, search_site='wikipedia.org'))
print(result)

3
[{'title': 'Banana - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Banana', 'body': '\n\nA banana is an elongated, edible fruit – botanically a berry[1][2] – produced by several kinds of large herbaceous flowering plants in the genus Musa.[3] In some countries, bananas used for cooking may be called "plantains", distinguishing them from dessert bananas. The fruit is variable in size, color, and firmness, but is usually elongated and curved, with soft flesh rich in starch covered with a rind, which may be green, yellow, red, purple, or brown when ripe. The fruits grow upward in clusters near the top of the plant. Almost all modern edible seedless (parthenocarp) bananas come from two wild species\xa0– Musa acuminata and Musa balbisiana. The scientific names of most cultivated bananas are Musa acuminata, Musa balbisiana, and Musa × paradisiaca for the hybrid Musa acuminata × M.\xa0balbisiana, depending on their genomic constitution. The old scientific name for this hybrid, Musa sap

In [10]:
web_search("what is apple")

[{'title': 'Apple Inc. | History, Products, Headquarters, & Facts',
  'href': 'https://www.britannica.com/topic/Apple-Inc',
  'body': 'Apple Inc., American manufacturer of personal computers, smartphones, tablet computers, computer peripherals, and computer software and one of the most recognizable brands in the world. It was the first successful personal computer company and the popularizer of the graphical user interface. Headquarters are located in Cupertino, California.'},
 {'title': 'Apple - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Apple',
  'body': 'An apple is a round, edible fruit produced by an apple tree ( Malus spp., among them the domestic or orchard apple; Malus domestica ). Apple trees are cultivated worldwide and are the most widely grown species in the genus Malus. The tree originated in Central Asia, where its wild ancestor, Malus sieversii, is still found.'}]

c:\Users\31477\.conda\envs\langchain\lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


OpenAI

In [10]:
import os

openai_api_key = ''
os.environ.update({"OPENAI_API_KEY": openai_api_key})

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)

In [117]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen1.5-0.5B"

max_new_tokens = 500
temperature = 0.1

device = 'cuda'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map=device
).to(device).eval()

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    # llama不支持fast
    use_fast=False if model.config.model_type == 'llama' else True
)

text = "Hello, who are you?"

def qwen_response(model, tokenizer, text):
    input_ids = tokenizer(text, return_tensors="pt", add_special_tokens=False).input_ids.to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids, max_new_tokens=max_new_tokens, do_sample=True,
            temperature=temperature
        )
    outputs = outputs.tolist()[0][len(input_ids[0]):]
    response = tokenizer.decode(outputs)
    response = response.strip()
    
    return response 

print(qwen_response(model, tokenizer, text))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


I'm a 15-year-old girl from the United States. I'm a student at a middle school. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Grade 7. I'm in Class 1, Gra

In [102]:
from typing import List, Optional, Any
from langchain_core.callbacks import (
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult

class QwenChatModel(BaseChatModel):
    """
    Simple Qwen Chat Model.
    这里有个比较矛盾的地方，记录一下。首先，BaseMessage 是一个 Union，类型为 str 或者 List(Union(str, dict))。但是在此处使用的时候，后文定义的正则处理函数指定了 model 的 output 类型强制为 str，所以这里把 output_str 写成 list 的形式就会报错。改成单条 string 了。
    """
    def messages_to_str(
        self,
        messages: List[BaseMessage],    
    ) -> List[str]:
        str_messages = []
        for msg in messages:
            if type(msg.content) == list:
                for m in msg.content:
                    str_messages.append(str(m))
            else:
                str_messages.append(str(msg.content))
        
        return str_messages

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        # output_str = []
        # for msg in self.messages_to_str(messages):
        #     response =  qwen_response(model, tokenizer, msg)# self._call(messages, stop=stop, run_manager=run_manager, **kwargs)
        #     output_str.append(response)
        # print(f"type:{type(output_str)}, content:{output_str}")
        # message = AIMessage(content=output_str)
        output_str = qwen_response(model, tokenizer, self.messages_to_str(messages)[0])
        message = AIMessage(content=output_str)
        generation = ChatGeneration(message=message)

        return ChatResult(generations=[generation])
    
    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "qwen-chat-model"

In [103]:
llm = QwenChatModel()

In [104]:
import re
from typing import List, Union

from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import LLMChain # from langchain.chains.base import Chain
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish # langchain_core.agents

In [105]:
def web_search_tool(search):
    result_list = web_search(search, search_engine='wikipedia', max_results=2)
    result = ""
    for i, r in enumerate(result_list):
        result += r['title']+"\n"+r['body']+"\n"
    return result

In [106]:
# Define which tools the agent can use to answer user queries
tools = [
    Tool(
        name="WebSearch",
        func=web_search_tool,
        description="useful for searching results from web",
    ),
]

In [107]:
template = """
Answer the following questions by searching informations on Internet. You need to figure out what knowledge you need to answer the questions and generate the corresponding queries to look up on the Internet using a search engine.
After you generate the corresponding query, you should search it. The results will be returned in string. 
Summarize results in natural language.

When generating query:
* Try to avoid any non-alphabetical symbols if possible
* Never end the query in question mark

You have access to the following tools:

{tools}

Use the following format:

Question: the input question for which you must provide a natural language answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}"""

In [108]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [109]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [110]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [111]:
output_parser = CustomOutputParser()
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [112]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [20]:
agent.invoke("what will happen if I throw a piece of sodium into water?")

AttributeError: 'LLMSingleActionAgent' object has no attribute 'invoke'

In [113]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [114]:
agent_executor.run("what will happen if I throw a piece of sodium into water?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.




> Entering new AgentExecutor chain...
Thought: the water will be affected by the sodium
Action: the action to take, should be one of [WebSearch]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: what will happen if I throw a piece of sodium into water?
Thought: the water will be affected by the sodium
Action: the action to take, should be one of [WebSearch]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: what will happen if I throw a piece of sodium into water?
Thought: the water will be affected by the sodium
Action: the action to take, should be one of [WebSearch]
Action Inpu

'the final answer to the original input question\n\nQuestion: what will happen if I throw a piece of sodium into water?\nThought: the water will be affected by the sodium\nAction: the action'

In [68]:
agent_executor.run("what will happen if I throw a piece of sodium into water?")



> Entering new AgentExecutor chain...
Thought: I need to find out what happens when sodium is put in water
Action: WebSearch
Action Input: "sodium reaction with water"

Observation:Sodium | Facts, Uses, & Properties | Britannica
sodium (Na), chemical element of the alkali metal group (Group 1 [Ia]) of the periodic table. Sodium is a very soft silvery-white metal. Sodium is the most common alkali metal and the sixth most abundant element on Earth, comprising 2.8 percent of Earth's crust. It occurs abundantly in nature in compounds, especially common salt —sodium ...
Sodium - Wikipedia
Many salts of sodium are highly water-soluble: sodium ions have been leached by the action of water from the Earth's minerals over eons, and thus sodium and chlorine are the most common dissolved elements by weight in the oceans. Sodium was first isolated by Humphry Davy in 1807 by the electrolysis of sodium hydroxide.

Sodium reacts with water to form sodium hydroxide and hydrogen gas
Final Answer: When

'When you throw a piece of sodium into water, it will react to form sodium hydroxide and hydrogen gas.'

c:\Users\31477\.conda\envs\langchain\lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [69]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a useful assistant with great amount of chemical knowledge."},
    {"role": "user", "content": "what will happen if I throw a piece of sodium into water?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="When you throw a piece of sodium into water, it will react vigorously with the water, producing hydrogen gas and heat. The reaction is so vigorous that it may even ignite the hydrogen gas, resulting in a fiery explosion. This is because sodium is a highly reactive metal, and it undergoes a violent exothermic reaction with water, forming sodium hydroxide and releasing hydrogen gas.\n\nIt's important to note that this is a highly dangerous experiment and should not be attempted without proper safety precautions and supervision. Sodium should only be handled by individuals with the necessary expertise and experience in handling reactive metals.", role='assistant', function_call=None, tool_calls=None)
